# Day of Week Analysis

This notebook analyzes how various health metrics vary by day of the week.

**Metrics analyzed:**
- Sleep Score
- Body Battery (max and min)
- Water Intake

**Analysis includes:**
- Average values by day of week (Sunday-first ordering)
- Statistical summaries (mean, median, standard deviation)
- Visualizations showing patterns across the week
- Identification of best and worst days for each metric


In [ ]:
# Import required libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import logging
from datetime import datetime

# Import our analysis functions
from garmin_analysis.features.day_of_week_analysis import (
    calculate_day_of_week_averages,
    plot_day_of_week_averages,
    print_day_of_week_summary,
    get_day_order
)
from garmin_analysis.utils.data_loading import load_master_dataframe

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configure plot style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10


## 1. Load Data

Load the master daily summary data containing all health metrics.


In [ ]:
# Load the master dataframe
logger.info("Loading master daily summary data...")
df = load_master_dataframe()

print(f"✓ Loaded {len(df)} days of data")
print(f"Date range: {df['day'].min()} to {df['day'].max()}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumns available: {', '.join(df.columns.tolist())}")


## 2. Calculate Day-of-Week Averages

Calculate average values for each metric by day of week.


In [ ]:
# Calculate day-of-week averages
day_averages = calculate_day_of_week_averages(df)

print(f"✓ Calculated averages for {day_averages['metric'].nunique()} metrics")
print(f"\nMetrics analyzed: {', '.join(day_averages['metric'].unique())}")

# Display the data
day_averages


## 3. Summary Statistics

View detailed statistics for each metric by day of week.


In [ ]:
# Print detailed summary
print_day_of_week_summary(df)


## 4. Visualizations

### 4.1 Individual Metric Plots


In [ ]:
# Create visualizations for each metric
day_order = get_day_order()
metrics = day_averages['metric'].unique()

for metric in metrics:
    metric_data = day_averages[day_averages['metric'] == metric].copy()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot 1: Bar chart with values
    bars = ax1.bar(metric_data['day_of_week'], metric_data['mean'], 
                   color=sns.color_palette("husl", len(metric_data)))
    ax1.set_title(f'{metric.replace("_", " ").title()} - Daily Averages', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Day of Week', fontsize=12)
    ax1.set_ylabel('Average Value', fontsize=12)
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars, metric_data['mean']):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{value:.1f}', ha='center', va='bottom', fontweight='bold')
    
    # Plot 2: Error bars (mean ± std)
    ax2.errorbar(range(len(metric_data)), metric_data['mean'], 
                yerr=metric_data['std'], fmt='o', markersize=8, capsize=5, capthick=2, linewidth=2)
    ax2.set_xticks(range(len(metric_data)))
    ax2.set_xticklabels(metric_data['day_of_week'], rotation=45)
    ax2.set_title(f'{metric.replace("_", " ").title()} - Mean ± Std Dev', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Day of Week', fontsize=12)
    ax2.set_ylabel('Value', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print best and worst days
    best_day = metric_data.loc[metric_data['mean'].idxmax()]
    worst_day = metric_data.loc[metric_data['mean'].idxmin()]
    
    print(f"\n{metric.replace('_', ' ').title()}:")
    print(f"  ✓ Best day:  {best_day['day_of_week']} ({best_day['mean']:.1f})")
    print(f"  ✗ Worst day: {worst_day['day_of_week']} ({worst_day['mean']:.1f})")
    print(f"  Δ Difference: {best_day['mean'] - worst_day['mean']:.1f}")
    print("-" * 50)


### 4.2 Combined Comparison


In [ ]:
# Create combined plot showing all metrics
fig, ax = plt.subplots(figsize=(14, 8))

# Pivot data for grouped bar chart
pivot_data = day_averages.pivot(index='day_of_week', columns='metric', values='mean')

# Create grouped bar chart
pivot_data.plot(kind='bar', ax=ax, width=0.8, colormap='viridis')
ax.set_title('Day-of-Week Averages: All Metrics Compared', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Day of Week', fontsize=13)
ax.set_ylabel('Average Value', fontsize=13)
ax.legend(title='Metrics', title_fontsize=12, fontsize=11, bbox_to_anchor=(1.05, 1), loc='upper left')
ax.tick_params(axis='x', rotation=45)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


### 4.3 Normalized Comparison

Scale all metrics to 0-100 range for easier comparison across different units.


In [ ]:
# Normalize each metric to 0-100 scale for comparison
normalized_data = day_averages.copy()
for metric in normalized_data['metric'].unique():
    mask = normalized_data['metric'] == metric
    values = normalized_data.loc[mask, 'mean']
    min_val = values.min()
    max_val = values.max()
    if max_val > min_val:
        normalized_data.loc[mask, 'normalized'] = 100 * (values - min_val) / (max_val - min_val)
    else:
        normalized_data.loc[mask, 'normalized'] = 50

# Create normalized comparison plot
fig, ax = plt.subplots(figsize=(14, 8))

pivot_normalized = normalized_data.pivot(index='day_of_week', columns='metric', values='normalized')

# Line plot
for column in pivot_normalized.columns:
    ax.plot(pivot_normalized.index, pivot_normalized[column], 
           marker='o', linewidth=2, markersize=8, label=column.replace('_', ' ').title())

ax.set_title('Normalized Day-of-Week Patterns (0-100 Scale)', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Day of Week', fontsize=13)
ax.set_ylabel('Normalized Value (0-100)', fontsize=13)
ax.legend(title='Metrics', title_fontsize=12, fontsize=11, bbox_to_anchor=(1.05, 1), loc='upper left')
ax.tick_params(axis='x', rotation=45)
ax.grid(True, alpha=0.3)
ax.set_ylim(-5, 105)

plt.tight_layout()
plt.show()


## 5. Key Insights

Generate a summary of key patterns and insights.


In [ ]:
print("="*70)
print("KEY INSIGHTS: DAY-OF-WEEK PATTERNS")
print("="*70)

for metric in day_averages['metric'].unique():
    metric_data = day_averages[day_averages['metric'] == metric]
    
    best_day = metric_data.loc[metric_data['mean'].idxmax()]
    worst_day = metric_data.loc[metric_data['mean'].idxmin()]
    overall_avg = metric_data['mean'].mean()
    overall_std = metric_data['std'].mean()
    
    print(f"\n📊 {metric.replace('_', ' ').title()}:")
    print(f"   Overall Average: {overall_avg:.1f} ± {overall_std:.1f}")
    print(f"   Best Performance: {best_day['day_of_week']} ({best_day['mean']:.1f})")
    print(f"   Worst Performance: {worst_day['day_of_week']} ({worst_day['mean']:.1f})")
    print(f"   Range: {best_day['mean'] - worst_day['mean']:.1f}")
    
    # Calculate coefficient of variation
    cv = (metric_data['mean'].std() / metric_data['mean'].mean()) * 100
    print(f"   Day-to-day Variability: {cv:.1f}%")
    
    # Find weekend vs weekday pattern
    weekend_days = ['Saturday', 'Sunday']
    weekend_avg = metric_data[metric_data['day_of_week'].isin(weekend_days)]['mean'].mean()
    weekday_avg = metric_data[~metric_data['day_of_week'].isin(weekend_days)]['mean'].mean()
    print(f"   Weekend Average: {weekend_avg:.1f}")
    print(f"   Weekday Average: {weekday_avg:.1f}")
    print(f"   Weekend Effect: {((weekend_avg - weekday_avg) / weekday_avg * 100):+.1f}%")

print("\n" + "="*70)


## 6. Export Results

Save the analysis results for future reference.


In [ ]:
# Save results to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"../reports/day_of_week_analysis_{timestamp}.csv"

day_averages.to_csv(output_file, index=False)
print(f"✓ Results saved to: {output_file}")

# Also create plot files
plot_files = plot_day_of_week_averages(df, save_plots=True, show_plots=False)
print(f"\n✓ Generated {len(plot_files)} visualization files:")
for metric, filepath in plot_files.items():
    print(f"  - {metric}: {filepath}")
